# TME9 - The Amplitude Damping (AD) code

In a previous TME, we saw different kind of error channels that can affect qubits. One of the most common error channel is the amplitude damping (AD) channel, which models energy dissipation from a quantum system to its environment. This type of error is particularly relevant in systems where qubits can lose energy over time, such as in superconducting qubits or trapped ions. It can also be the loss of photon, pure loss or fiber attenuation in optical quantum communication.

Quantum error correction codes are designed to protect quantum information from various types of errors, including amplitude damping. The AD code is a specific quantum error-correcting code that is tailored to correct errors caused by the amplitude damping channel. These errors will be described using a Pauli $(I, X, Y, Z)$ basis.

## The Amplitude Damping Channel

A single qubit quantum noise process is described by:

$$\mathcal{E}(\rho) = \sum_k E_k \rho E_k^\dagger,$$

where the Kraus operators $E_k$ satisfy the completeness relation $\sum_k E_k^\dagger E_k = I$. For the amplitude damping channel with damping probability $\gamma$, the Kraus operators are given by:

$$E_0 = \begin{pmatrix} 1 & 0 \\ 0 & \sqrt{1 - \gamma} \end{pmatrix}, \quad E_1 = \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0 \end{pmatrix}.$$

The fundamental idea is to use several qubits to encode a single logical qubit in such a way that if one of the physical qubits undergoes amplitude damping, the original logical qubit can still be recovered.

**Question:** Using the Bloch representation $\rho = \frac{1}{2}(I + xX + yY + zZ)$, what is the effect of the amplitude damping channel on the Bloch vector $(x, y, z)$? You'll provide a mathematical expression of the new Bloch vector $(x', y', z')$ as a function of $(x, y, z)$ and $\gamma$ and define a function that takes the Bloch vector and $\gamma$ as inputs and returns the new Bloch vector.

## Encoding the logical qubit

Let's say we use the following 4 qubit encoding for the logical qubit:

$$|0_L\rangle = \frac{1}{\sqrt{2}} (|0000\rangle + |1111\rangle),$$
$$|1_L\rangle = \frac{1}{\sqrt{2}} (|0011\rangle + |1100\rangle).$$

**Question 1:** Are these states orthogonal? Justify your answer.

**Question 2:** Design a circuit that takes a single qubit $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ and encodes it into the logical qubit state $|\psi_L\rangle = \alpha_L|0_L\rangle + \beta_L|1_L\rangle$. Define a function `encoding_circuit(alpha, beta)` that returns the corresponding $\Psi_L$ statevector.

 #### Effect of Amplitude Damping on the Bloch Vector

 The density matrix of a single qubit in the Bloch sphere representation is:

 $$
 \rho = \frac{1}{2}(I + xX + yY + zZ) =
 \begin{pmatrix}
 \frac{1 + z}{2} & \frac{x - i y}{2} \\
 \frac{x + i y}{2} & \frac{1 - z}{2}
 \end{pmatrix}
 $$

 The amplitude damping channel acts as:

 $$
 \mathcal{E}(\rho) = E_0 \rho E_0^\dagger + E_1 \rho E_1^\dagger
 $$

 where the Kraus operators are:

 $$
 E_0 =
 \begin{pmatrix}
 1 & 0 \\
 0 & \sqrt{1-\gamma}
 \end{pmatrix}, \quad
 E_1 =
 \begin{pmatrix}
 0 & \sqrt{\gamma} \\
 0 & 0
 \end{pmatrix}
 $$

 Applying this channel results in the transformation:

 $$
 \rho' =
 \begin{pmatrix}
 \rho_{00} + \gamma \rho_{11} & \sqrt{1-\gamma}\, \rho_{01} \\
 \sqrt{1-\gamma}\, \rho_{10} & (1-\gamma) \rho_{11}
 \end{pmatrix}
 $$

 With $\rho_{01} = \frac{x - i y}{2}$ and $\rho_{11} = \frac{1-z}{2}$, we can extract the new Bloch sphere coordinates:

 $$
 \begin{aligned}
 x' &= \sqrt{1-\gamma}\; x \\
 y' &= \sqrt{1-\gamma}\; y \\
 z' &= (1-\gamma)\, z + \gamma
 \end{aligned}
 $$

 A convenient implementation in Python is:

 ```python
 import math

 def amplitude_damping_bloch(x, y, z, gamma):
     s = math.sqrt(1 - gamma)
     return (s * x, s * y, (1 - gamma) * z + gamma)
 ```


 ## Encoding the Logical Qubit

 ### Question 1: Orthogonality

 Given:

 $$
 |0_L\rangle = \frac{1}{\sqrt{2}}(|0000\rangle + |1111\rangle),\quad
 |1_L\rangle = \frac{1}{\sqrt{2}}(|0011\rangle + |1100\rangle)
 $$

 Their inner product is:

 $$
 \langle 0_L|1_L\rangle = \frac{1}{2} \Big(
 \langle 0000|0011\rangle +
 \langle 0000|1100\rangle +
 \langle 1111|0011\rangle +
 \langle 1111|1100\rangle
 \Big) = 0
 $$

 because all computational basis states appearing are distinct and mutually orthogonal. Therefore, $|0_L\rangle$ and $|1_L\rangle$ are orthogonal.

 ### Question 2: An Encoding Circuit and Statevector Function

 Let the four qubits be $(q_0, q_1, q_2, q_3)$ with the input state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ on $q_0$, and ancillas $|000\rangle$ on $q_1, q_2, q_3$.

 One encoding circuit that implements $|\psi\rangle|000\rangle \mapsto \alpha|0_L\rangle + \beta|1_L\rangle$ is:

 1. **SWAP** $(q_0 \leftrightarrow q_2)$ (move $|\psi\rangle$ onto $q_2$; now $q_0$ is $|0\rangle$)
 2. Apply $H$ (Hadamard) to $q_0$
 3. Apply $\mathrm{CNOT}(q_0 \rightarrow q_1)$
 4. Apply $\mathrm{CNOT}(q_2 \rightarrow q_3)$
 5. Apply $\mathrm{CNOT}(q_0 \rightarrow q_2)$
 6. Apply $\mathrm{CNOT}(q_0 \rightarrow q_3)$

 This produces exactly:

 $$
 |\psi_L\rangle =
 \frac{\alpha}{\sqrt{2}}(|0000\rangle + |1111\rangle) +
 \frac{\beta}{\sqrt{2}}(|0011\rangle + |1100\rangle)
 $$


In [8]:
# Code here !
import numpy as np
import math

def encoding_circuit(alpha, beta):
    # Basis order: |0000>, |0001>, ..., |1111> with q0 as MSB, q3 as LSB.
    psiL = np.zeros(16, dtype=complex)
    a = alpha / math.sqrt(2)
    b = beta  / math.sqrt(2)

    psiL[0b0000] += a
    psiL[0b1111] += a
    psiL[0b0011] += b
    psiL[0b1100] += b
    return psiL


## Applying the Amplitude Damping Channel

Now, let's consider the effect of the amplitude damping channel on each of the four qubits in the encoded state. We will apply the amplitude damping channel independently to each qubit.

**Question 1:** Define a function `apply_amplitude_damping(statevector, gamma)` that takes the encoded statevector and the damping probability $\gamma$ as inputs and returns the new statevector after applying the amplitude damping channel to each qubit.

**Question 2:** For each index $i$ in $\{0, 1, 2, 3\}$, compute the statevector after applying the amplitude damping channel to only the $i$-th qubit. Store these state vectors in a list called `damped_states`.

**Question 3:** Does the states in `damped_states` form an orthogonal set? Justify wer answer.

What we did here is to simulate the effect of amplitude damping errors occurring on each qubit of the encoded state. We want now to correct these errors and retrieve the original logical qubit.

In [9]:
# Code here !
import numpy as np
import itertools

def _kraus_ops(gamma):
    E0 = np.array([[1.0, 0.0],
                   [0.0, np.sqrt(1.0 - gamma)]], dtype=complex)
    E1 = np.array([[0.0, np.sqrt(gamma)],
                   [0.0, 0.0]], dtype=complex)
    return [E0, E1]

def _apply_single_qubit_op(statevector, op, qubit_index, n_qubits=4):
    """
    Applies a 2x2 operator `op` to `qubit_index` (0 = most significant qubit q0)
    on an n_qubits statevector of length 2^n.
    """
    psi = np.asarray(statevector, dtype=complex).reshape([2]*n_qubits)

    # tensordot op (out,in) with psi axis=qubit_index (in)
    out = np.tensordot(op, psi, axes=([1], [qubit_index]))
    # out axes are: (op_out, psi_axes_without_target). Put back to original order.
    axes = list(range(1, n_qubits))               # psi axes after removing target
    out = np.moveaxis(out, 0, qubit_index)        # place op_out into target position

    return out.reshape(-1)

def apply_amplitude_damping(statevector, gamma):
    """
    Applies amplitude damping independently to all 4 qubits.

    Returns:
      rho_out: 16x16 density matrix after the channel
      branches: dict mapping outcome bits (k0,k1,k2,k3) to unnormalized statevectors
    """
    E = _kraus_ops(gamma)

    branches = {}
    rho_out = np.zeros((16, 16), dtype=complex)

    for ks in itertools.product([0, 1], repeat=4):
        psi = np.asarray(statevector, dtype=complex)
        for q, k in enumerate(ks):
            psi = _apply_single_qubit_op(psi, E[k], qubit_index=q, n_qubits=4)
        branches[ks] = psi
        rho_out += np.outer(psi, np.conjugate(psi))

    return rho_out, branches

# Question 2
def apply_amplitude_damping_to_one_qubit(statevector, gamma, i):
    E = _kraus_ops(gamma)

    branches = []
    rho_out = np.zeros((16, 16), dtype=complex)

    for k in [0, 1]:
        psi_k = _apply_single_qubit_op(statevector, E[k], qubit_index=i, n_qubits=4)
        branches.append(psi_k)
        rho_out += np.outer(psi_k, np.conjugate(psi_k))

    return rho_out, branches  # branches are the (E0 on i) and (E1 on i) contributions

# --- Example input state and damping strength (needed for the cells below) ---
# Pick any normalized (alpha, beta) and a small gamma.
# (This guard lets this cell run even if you didn't execute the earlier encoding cell.)
if 'encoding_circuit' not in globals():
    def encoding_circuit(alpha, beta):
        # Basis order: |0000>, |0001>, ..., |1111> with q0 as MSB, q3 as LSB.
        psiL = np.zeros(16, dtype=complex)
        a = alpha / np.sqrt(2)
        b = beta  / np.sqrt(2)
        psiL[0b0000] += a
        psiL[0b1111] += a
        psiL[0b0011] += b
        psiL[0b1100] += b
        return psiL

gamma = 0.05
alpha = 1 / np.sqrt(2)
beta = 1 / np.sqrt(2)
statevector = encoding_circuit(alpha, beta)

# damped_states: store the density matrices after damping only qubit i
damped_states = []
for i in [0, 1, 2, 3]:
    rho_i, _ = apply_amplitude_damping_to_one_qubit(statevector, gamma, i)
    damped_states.append(rho_i)



#### Question 3: are the states in damped_states orthogonal?

If damped_states contains the full channel outputs (density matrices) for each 𝑖: 
“orthogonality” is not the right notion (they are mixed states). We can instead
compare supports / subspaces or use state distinguishability measures (trace 
distance, fidelity).

If damped_states contains the post-jump vectors ∣𝜓𝑖⟩ ∝ (𝐸1 on qubit 𝑖)∣𝜓𝐿⟩ : then 
for this specific code, the single-jump states lie in mutually orthogonal 
computational basis patterns (a different qubit flips from ∣1⟩→∣0⟩ in the ∣1111⟩ 
component), so they are orthogonal across different 𝑖. Concretely, the 
“jumped” basis components are ∣0111⟩, ∣1011⟩, ∣1101⟩, ∣1110⟩, which are orthonormal 
and therefore distinguishable by a measurement.

This orthogonality of the single-jump error states is exactly what we leverage for error detection/correction. We can identify which qubit damped (which syndrome) and then attempt recovery.

## Error Correction

To correct the errors introduced by the amplitude damping channel, we will perform a series of measurements and apply corrective operations based on the measurement outcomes. First, we define the projectors $ P = |0_L\rangle\langle0_L| + |1_L\rangle\langle1_L|, $ and $ P_j $ projector onto the subspace where the j-th qubit has undergone amplitude damping $\{E_1^{(j)}|0_L\rangle, E_1^{(j)}|1_L\rangle\} $.

**Question 1:** Verify that $ P + \sum_{j=0}^{3} P_j = I $, where $I$ is the identity operator on the 4-qubit space. And that they are indeed projectors.

**Question 2:** We consider $\gamma$ small such that we can do a first order approximation. Write the approximate expression of $E_0$ and $E_1$ at first order in $\gamma$.

We will build a recovery operations $R_j$ such that $ R_j E_1^{(j)}|0_L\rangle = |0_L\rangle, \quad R_j E_1^{(j)}|1_L\rangle = |1_L\rangle $:

$$ R_j : \text{span}\{E_1^{(j)}|0_L\rangle, E_1^{(j)}|1_L\rangle\} \rightarrow \text{span}\{|0_L\rangle, |1_L\rangle\}. $$

**Tasks:**
1. Recall the actions of $E_1^{(j)}$ on $|0_L\rangle$ and $|1_L\rangle$. What forms do the resulting states take? Do they form an orthogonal set? Note the resulting vectors (normalised) $|\tilde{\phi}_{0,1}^{(j)}\rangle$.
2. The goal is to map $|\tilde{\phi}_{0}^{(j)}\rangle$ to $|0_L\rangle$ and $|\tilde{\phi}_{1}^{(j)}\rangle$ to $|1_L\rangle$. To achieve this, we can use a unitary transformation that performs this mapping. Propose an operator $R_j$ that achieves this transformation.
3. Define a function `recovery_operator(j)` that returns the recovery operator $R_j$ for the j-th qubit.
4. Using the recovery operators, define a function `error_correction(statevector)` that applies the appropriate recovery operation based on the measurement outcomes and returns the corrected statevector. Take the example of $E_1^{(j)}\Psi_L$.
5. We left $E_0$ errors aside for now. Discuss why this is a reasonable approximation when $\gamma$ is small.

### Question 1: Projectors and (restricted) completeness

Define the **code space**:
$$
\begin{align*}
\mathcal{C} &= \mathrm{span}\{|0_L\rangle, |1_L\rangle\}, \\
P &= |0_L\rangle\langle 0_L| + |1_L\rangle\langle 1_L|.
\end{align*}
$$

For each $j \in \{0,1,2,3\}$, define the **single-jump error space**:
$$
\mathcal{E}_j = \mathrm{span}\left\{ E_1^{(j)}|0_L\rangle,\, E_1^{(j)}|1_L\rangle \right\}
$$
and let $P_j$ be the projector onto $\mathcal{E}_j$.

#### (a) They are projectors

- $P^2 = P$, $P^\dagger = P$ because $|0_L\rangle, |1_L\rangle$ are orthonormal.
- If $\{ |\tilde\phi_0^{(j)}\rangle, |\tilde\phi_1^{(j)}\rangle \}$ is an orthonormal basis of $\mathcal{E}_j$, then
  $$
  P_j = |\tilde\phi_0^{(j)}\rangle\langle \tilde\phi_0^{(j)}| + |\tilde\phi_1^{(j)}\rangle\langle \tilde\phi_1^{(j)}|
  $$
  satisfies $P_j^2 = P_j$ and $P_j^\dagger = P_j$.

#### (b) Orthogonality and (restricted) completeness

For this code, $\mathcal{C} \perp \mathcal{E}_j$ and $\mathcal{E}_j \perp \mathcal{E}_k$ for $j \neq k$ (each is spanned by different computational basis states; explicit forms are below). Hence,
$$
P P_j = 0, \quad P_j P_k = 0 \quad (j \neq k).
$$

However, $\dim(\mathcal{C}) = 2$ and $\dim(\mathcal{E}_j) = 2$, so
$$
\dim\left( \mathcal{C} \oplus \bigoplus_{j=0}^3 \mathcal{E}_j \right) = 2 + 4 \cdot 2 = 10 \neq 16.
$$
So
$$
P + \sum_{j=0}^{3} P_j \neq I
$$
on the full 4-qubit Hilbert space; it is true as an identity **on the 10-dimensional subspace**
$$
\mathcal{H}_{\leq 1\, \mathrm{jump}} := \mathcal{C} \oplus \bigoplus_{j=0}^3 \mathcal{E}_j,
$$
i.e., it resolves the identity on the “no jump + exactly one jump” sector, which is the relevant sector at first order in $\gamma$.

### Question 2: First-order approximation of $E_0, E_1$

Use $\sqrt{1-\gamma} = 1 - \gamma/2 + O(\gamma^2)$. Then
$$
E_0 = \begin{pmatrix}1 & 0 \\ 0 & \sqrt{1-\gamma}\end{pmatrix}
\approx
\begin{pmatrix}1 & 0 \\ 0 & 1-\gamma/2\end{pmatrix}
= I - \frac{\gamma}{2} |1\rangle\langle 1| + O(\gamma^2).
$$
And
$$
E_1 = \begin{pmatrix}0 & \sqrt{\gamma} \\ 0 & 0\end{pmatrix}
= \sqrt{\gamma}\, |0\rangle\langle 1|.
$$
($E_1$ is order $\sqrt{\gamma}$, while its **probability** contribution $E_1 \rho E_1^\dagger$ is order $\gamma$.)


#### Tasks 1–2: Compute $E_1^{(j)}|0_L\rangle$, $E_1^{(j)}|1_L\rangle$ and define normalized $|\tilde\phi\rangle$

Recall $E_1|0\rangle = 0,\, E_1|1\rangle = \sqrt{\gamma}\,|0\rangle$.

##### Action on $|0_L\rangle = \frac{1}{\sqrt{2}}(|0000\rangle + |1111\rangle)$

Only $|1111\rangle$ contributes:
$$
E_1^{(j)}|0_L\rangle = \frac{\sqrt{\gamma}}{\sqrt{2}}\,|v_j\rangle,
$$
where (qubit order $q_0 q_1 q_2 q_3$):
$$
\begin{aligned}
|v_0\rangle &= |0111\rangle \\
|v_1\rangle &= |1011\rangle \\
|v_2\rangle &= |1101\rangle \\
|v_3\rangle &= |1110\rangle
\end{aligned}
$$

##### Action on $|1_L\rangle = \frac{1}{\sqrt{2}}(|0011\rangle + |1100\rangle)$

Only the component with a 1 at position $j$ contributes:
$$
E_1^{(j)}|1_L\rangle = \frac{\sqrt{\gamma}}{\sqrt{2}}\,|w_j\rangle,
$$
with
$$
\begin{aligned}
|w_0\rangle &= |0100\rangle \\
|w_1\rangle &= |1000\rangle \\
|w_2\rangle &= |0001\rangle \\
|w_3\rangle &= |0010\rangle
\end{aligned}
$$

Thus the **normalized** error-basis vectors are simply
$$
|\tilde\phi_0^{(j)}\rangle = |v_j\rangle,\qquad
|\tilde\phi_1^{(j)}\rangle = |w_j\rangle.
$$
They form an orthonormal set (all are distinct computational basis states), so each $\mathcal{E}_j$ is a clean 2D orthogonal syndrome subspace.


## Task 2: Propose a recovery operator $R_j$

On the error subspace $\mathcal{E}_j = \mathrm{span}\{|v_j\rangle, |w_j\rangle\}$, define the partial isometry
$$
R_j = |0_L\rangle\langle v_j| + |1_L\rangle\langle w_j|.
$$
Then
$$
R_j|v_j\rangle = |0_L\rangle, \qquad R_j|w_j\rangle = |1_L\rangle,
$$
and
$$
R_j^\dagger R_j = |v_j\rangle\langle v_j| + |w_j\rangle\langle w_j| = P_j, \qquad
R_j R_j^\dagger = P.
$$
As written, $R_j$ is not a full $16 \times 16$ unitary; it is the correct recovery **Kraus operator** conditioned on syndrome $j$. We can extend it arbitrarily on the orthogonal complement to make a unitary if you want.


In [10]:
# Code here !

# Task 3
def _basis_state(idx, n=4):
    v = np.zeros(2**n, dtype=complex)
    v[idx] = 1.0
    return v

def _logical_codewords():
    # |0_L> = (|0000> + |1111>)/sqrt(2)
    # |1_L> = (|0011> + |1100>)/sqrt(2)
    s2 = math.sqrt(2)
    zeroL = (_basis_state(0b0000) + _basis_state(0b1111)) / s2
    oneL  = (_basis_state(0b0011) + _basis_state(0b1100)) / s2
    return zeroL, oneL

def recovery_operator(j):
    zeroL, oneL = _logical_codewords()

    v_idx = [0b0111, 0b1011, 0b1101, 0b1110][j]  # |v_j>
    w_idx = [0b0100, 0b1000, 0b0001, 0b0010][j]  # |w_j>

    vj = _basis_state(v_idx)
    wj = _basis_state(w_idx)

    # R_j = |0_L><v_j| + |1_L><w_j|
    Rj = np.outer(zeroL, np.conjugate(vj)) + np.outer(oneL, np.conjugate(wj))
    return Rj

# Task 4
def _projector_from_vectors(vectors):
    # vectors should be orthonormal
    P = np.zeros((16,16), dtype=complex)
    for v in vectors:
        P += np.outer(v, np.conjugate(v))
    return P

def _syndrome_projectors():
    zeroL, oneL = _logical_codewords()
    P = _projector_from_vectors([zeroL, oneL])

    Pjs = []
    v_list = [0b0111, 0b1011, 0b1101, 0b1110]
    w_list = [0b0100, 0b1000, 0b0001, 0b0010]
    for j in range(4):
        vj = _basis_state(v_list[j])
        wj = _basis_state(w_list[j])
        Pjs.append(_projector_from_vectors([vj, wj]))
    return P, Pjs

def error_correction(statevector):
    psi = np.asarray(statevector, dtype=complex)
    P, Pjs = _syndrome_projectors()

    # compute weights
    w0 = np.vdot(psi, P @ psi).real
    wj = [np.vdot(psi, Pjs[j] @ psi).real for j in range(4)]

    # choose syndrome: code space vs one of the jump spaces
    best = np.argmax([w0] + wj)

    if best == 0:
        # already in code space (no-jump sector)
        psi_corr = P @ psi
    else:
        j = best - 1
        psi_proj = Pjs[j] @ psi
        Rj = recovery_operator(j)
        psi_corr = Rj @ psi_proj

    # renormalize
    norm = np.linalg.norm(psi_corr)
    if norm > 0:
        psi_corr = psi_corr / norm
    return psi_corr



 #### Task 5: Why ignoring $E_0$ is reasonable when $\gamma$ is small
 
 At small $\gamma$:
 
 * A “jump” ($E_1$) occurs with probability $O(\gamma)$ and pushes the state into one of the **orthogonal** syndrome subspaces $\mathcal{E}_j$, which is correctable by a projective syndrome measurement plus $R_j$.
 * “No jump” ($E_0$) occurs with probability $1 - O(\gamma)$, and $E_0 \approx I - \frac{\gamma}{2}|1\rangle\langle 1|$ causes only a **small distortion** (amplitude shrink on components containing $|1\rangle$). This distortion does not move the state into an orthogonal syndrome subspace in the same clean way, so correcting it requires a more refined *approximate* QEC treatment.
 
 Thus, to first order and for illustrating the mechanism, focusing on $E_1$ errors captures the dominant *detectable/correctable* event structure, while $E_0$ is treated as a small residual error.


## Why do we need several qubits to correct AD errors?

The amplitude damping (AD) channel represents a type of error that causes a qubit to lose energy, typically transitioning from the excited state $|1\rangle$ to the ground state $|0\rangle$. This type of error is particularly challenging to correct because it is not simply a bit-flip or phase-flip error; instead, it involves a loss of information about the qubit's state. Amplitude damping is not a unitary error, meaning it cannot be reversed by a simple unitary operation.

The underlying question is not why 4 qubits are needed specifically, but rather how many orthogonal subspaces must the code be able to distinguish in order to correct for the possible errors introduced by the amplitude damping channel. The action of AD on each qubit must land in distinct subspaces that can be identified and corrected:

$$ \mathcal{C} \oplus \mathcal{E}_0 \oplus \mathcal{E}_1 \oplus \mathcal{E}_2 \oplus \mathcal{E}_3, $$

where $\mathcal{C}$ is the codespace $\text{span}\{|0_L\rangle, |1_L\rangle\}$ and $\mathcal{E}_j$ is the error space associated with an amplitude damping error on the j-th qubit, carrying two orthogonal states $\text{span}\{E_1^{(j)}|0_L\rangle, E_1^{(j)}|1_L\rangle\} $.

So we need at least 1 codespace of dimension 2 (to encode the logical qubit). But why at least 4 error spaces of dimension 2?
Each qubit can either remain unaffected or undergo an amplitude damping error. For a 4-qubit code, there are 4 possible single-qubit amplitude damping errors (one for each qubit). Each of these errors maps the logical states into distinct orthogonal subspaces. Thus, to correct for all possible single-qubit amplitude damping errors, we need to be able to distinguish between the codespace and these 4 error spaces. Each error space must also be able to accommodate the two orthogonal states resulting from the amplitude damping of the logical qubit states.

**Question:** Try to apply the method developed in this TME for a 1, 2 and 3 qubit encoding. Show that it is not possible to correct for amplitude damping errors with less than 4 qubits. With $n$ qubits, how many orthogonal subspaces of dimension 2 can you create? What is the size of the total Hilbert space? What condition must then be satisfied to be able to correct for AD errors?

_To answer, you may either use mathematical reasoning for n=1 as it is the simplest case. The idea is to write a code that tests all possible orthogonal encodings for n=2 and n=3 qubits, which is faster than doing it by hand._

In [11]:
# Code here !

def sigma_minus_on_qubit(n, j):
    sm = np.array([[0, 1],
                   [0, 0]], dtype=complex)
    I = np.eye(2, dtype=complex)
    ops = [I]*n
    ops[j] = sm
    op = ops[0]
    for k in range(1, n):
        op = np.kron(op, ops[k])
    return op

def random_codewords(dim):
    A = (np.random.randn(dim, 2) + 1j*np.random.randn(dim, 2))
    Q, _ = np.linalg.qr(A)  # columns orthonormal
    return Q[:,0], Q[:,1]

def kl_violation(n, v0, v1):
    # error set: {I, sigma^-_0,...,sigma^-_{n-1}}
    dim = 2**n
    E = [np.eye(dim, dtype=complex)] + [sigma_minus_on_qubit(n, j) for j in range(n)]

    # KL wants <0|Ea†Eb|1>=0 and <0|Ea†Eb|0> = <1|Ea†Eb|1> for all a,b.
    viol = 0.0
    for Ea in E:
        for Eb in E:
            M = Ea.conj().T @ Eb
            off = np.vdot(v0, M @ v1)                       # should be 0
            diag0 = np.vdot(v0, M @ v0)
            diag1 = np.vdot(v1, M @ v1)
            viol += (abs(off)**2 + abs(diag0 - diag1)**2)
    return float(np.real(viol))

def search_min_violation(n, trials=20000, seed=0):
    np.random.seed(seed)
    dim = 2**n
    best = 1e99
    best_pair = None
    for _ in range(trials):
        v0, v1 = random_codewords(dim)
        val = kl_violation(n, v0, v1)
        if val < best:
            best = val
            best_pair = (v0, v1)
    return best, best_pair

# Example usage:
print(search_min_violation(2, trials=5000)[0])  # impossible by dimension
print(search_min_violation(3, trials=50000)[0]) # stays > 0 (no exact code found)


0.6318539765184411
0.7884996699242361


**Answer here !**

### Counting argument (necessary condition)
 
 With $n$ physical qubits the Hilbert space has dimension
 $$
 \dim(\mathcal{H}) = 2^n.
 $$
 
 If we want to correct **a single amplitude-damping “jump”** on any one qubit, we need to distinguish (at least) the mutually orthogonal syndrome subspaces
 $$
 \mathcal{C}\;\oplus\;\mathcal{E}_0\;\oplus\cdots\oplus\;\mathcal{E}_{n-1},
 $$
 where
 
 * $\dim(\mathcal{C})=2$ (one logical qubit),
 * each $\mathcal{E}_j=\mathrm{span}\{E_1^{(j)}|0_L\rangle,\,E_1^{(j)}|1_L\rangle\}$ must have $\dim(\mathcal{E}_j)=2$ to preserve the logical information after the error.
 
 So we need room for at least $n+1$ **orthogonal 2D subspaces**, i.e.
 $$
 2(n+1)\ \le\ 2^n
 \quad\Longleftrightarrow\quad
 n+1\ \le\ 2^{n-1}.
 $$
 Equivalently: the maximum number of pairwise orthogonal 2D subspaces you can pack into $2^n$ dimensions is
 $$
 \left\lfloor \frac{2^n}{2}\right\rfloor = 2^{n-1},
 $$
 and we need at least $n+1$ of them.
 
 This immediately gives:
 
 * $n=1$: $2(n+1)=4>2^1=2$ impossible.
 * $n=2$: $2(n+1)=6>2^2=4$ impossible.
 * $n=3$: $2(n+1)=8=2^3$ **dimension-counting does not rule it out** (it is only a necessary condition).
 
 So: $n\ge 3$ is necessary, but not sufficient.
 
 ### Why $n=3$ still fails for amplitude damping (the real obstruction)
 
 For amplitude damping, the relevant single-jump error operators are $E_1^{(j)} \propto \sigma^-_j$ (lowering on qubit $j$). Exact correction requires the Knill–Laflamme conditions for the set $\{I,\,\sigma^-_0,\ldots,\sigma^-_{n-1}\}$:
 $$
 \langle i_L| (\sigma^-_a)^\dagger \sigma^-_b |j_L\rangle = C_{ab}\,\delta_{ij}
 \quad\text{for }i,j\in\{0,1\}.
 $$
 Even though $n=3$ passes the dimension bound, these quadratic constraints on the codewords turn out to be infeasible: there is **no** 3-qubit code whose syndrome spaces for $\sigma^-_0,\sigma^-_1,\sigma^-_2$ are simultaneously (i) two-dimensional (for both logical states), (ii) mutually orthogonal, and (iii) orthogonal to the codespace. In other words, dimension counting says “maybe”, but the *structure* of $\sigma^-_j$ says “no”.
 
 
 ### A practical brute-force test for $n=2$ and $n=3$
 
 
 Interpretation:
 
 * For $n=2$, you already know it’s impossible because $2(n+1)>2^n$.
 * For $n=3$, if you run enough trials, you will find the best objective stays clearly $>0$, consistent with “no exact 3-qubit AD code”.
 * For $n=4$, if you plug in your known 4-qubit codewords ($|0_L\rangle,|1_L\rangle$), the objective drops to (numerically) $\sim$0 for the single-jump error model.
 

 * Total Hilbert space size: $\dim(\mathcal{H}) = 2^n$.
 * Max number of orthogonal 2D subspaces: $2^{n-1}$.
 * To correct single-qubit AD “jumps” via orthogonal syndrome spaces of dimension 2:
   $$
   n+1 \le 2^{n-1}\quad\text{(necessary).}
   $$
 * This rules out $n=1,2$. $n=3$ passes the count but fails the AD-specific Knill–Laflamme constraints; $n=4$ is the smallest where an exact single-jump-correcting construction exists in this framework.
